# 고양이 VS 개 이진분류 모델

- https://www.kaggle.com/competitions/dogs-vs-cats-redux-kernels-edition/overview

# Import

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings("ignore")
# 실행에 필요한 메시지 외 경고가 출력되지 않게 해줌
import os

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from tensorflow import keras

In [6]:
from sklearn.model_selection import train_test_split

# DataSet

In [7]:
imageDataGenerator = ImageDataGenerator(
    rescale=1./255,                  # 이건 그냥 고정 이미지 - 픽셀값을 0~255에서 0~1범위로 변경
    shear_range=0.2,                 # 기울기 범위
    zoom_range=0.2,                  # 확대 범위
    horizontal_flip=True,           # 상하반전
    validation_split=0.1            # 배치비율(예: 0.3, 0.2, 0.25 등등)
)
imageDataGenerator_test = ImageDataGenerator(rescale=1./255)

train_dataset = imageDataGenerator.flow_from_directory(
    "./data/train/",
    target_size=(128,128),
    batch_size=100,
    subset="training",
    class_mode="binary"
)

val_dataset = imageDataGenerator.flow_from_directory(
    "./data/train/",
    target_size=(128,128),
    batch_size=100,
    subset="validation",
    class_mode="binary"
)

test_dataset = imageDataGenerator_test.flow_from_directory(
    "./data/test/",
    target_size=(128,128),
    batch_size=100,
    shuffle=False
)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


# Function

In [8]:
# 예측값을 넣으면 파일 순서에 맞게 정렬해서 반환
def get_labels(preds):
    # test 폴더에서 파일 목록 가져오기
    path = "data/test/test"
    files = os.listdir(path)
    sorted_files = sorted(files)
    
    # 파일이름을 알파벳순이 아닌 정수 오름차순으로 변경하기 위한 작업
    df = pd.DataFrame()
    df["filename"] = sorted_files
    df["pred"] = preds
    df["filename"] = df["filename"].str.replace(".jpg", "", regex=False)
    df["filename"] = df["filename"].astype(int)
    df = df.sort_values("filename")
    return df["pred"].values

In [9]:
# 파일을 저장하는 함수
# save_preds(예측결과, 파일명) 으로 실행하면 된다
def save_preds(preds, file_name=None):
    # 파일 저장
    if file_name is None:
        file_name = f"dog_vs_cat_submission_{get_filename()}.csv"
    submission_df = pd.read_csv("submission/sample_submission.csv", index_col="id")
    submission_df["label"] = get_labels(preds)
    submission_df.to_csv("submission/" + file_name)
    
    print(file_name + " Saved!")

In [10]:
import matplotlib.pyplot as plt

def show_graph(history_dict):
    accuracy = history_dict['acc']
    val_accuracy = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = range(1, len(loss) + 1)
    
    plt.figure(figsize=(16, 1))
    
    plt.subplot(121)
    plt.subplots_adjust(top=2)
    plt.plot(epochs, accuracy, 'ro', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
    plt.title('Trainging and validation accuracy and loss')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy and Loss')

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5)
#     plt.legend(bbox_to_anchor=(1, -0.1))

    plt.subplot(122)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=5)
#     plt.legend(bbox_to_anchor=(1, 0))

    plt.show()

# Modeling(1)

- 사이즈 255,255 batch = 10

In [112]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (255,255, 3)))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 16384)             0         
                                                                 
 dense_18 (Dense)            (None, 32)              

In [113]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-2model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [87]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [88]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("1model.h5")

Epoch 1/100
2250/2250 [==============================] - 518s 230ms/step - loss: 0.6236 - accuracy: 0.6470 - val_loss: 0.5380 - val_accuracy: 0.7300
Epoch 2/100
2250/2250 [==============================] - 935s 415ms/step - loss: 0.5536 - accuracy: 0.7156 - val_loss: 0.5198 - val_accuracy: 0.7472
Epoch 3/100
2250/2250 [==============================] - 796s 354ms/step - loss: 0.5341 - accuracy: 0.7321 - val_loss: 0.5276 - val_accuracy: 0.7504
Epoch 4/100
2250/2250 [==============================] - 489s 217ms/step - loss: 0.5188 - accuracy: 0.7463 - val_loss: 0.4785 - val_accuracy: 0.7608
Epoch 5/100
2250/2250 [==============================] - 486s 216ms/step - loss: 0.5051 - accuracy: 0.7521 - val_loss: 0.4746 - val_accuracy: 0.7720
Epoch 6/100
2250/2250 [==============================] - 487s 216ms/step - loss: 0.4940 - accuracy: 0.7629 - val_loss: 0.4583 - val_accuracy: 0.7872
Epoch 7/100
2250/2250 [==============================] - 487s 217ms/step - loss: 0.4867 - accuracy: 0.7688

# model(2) - 사이즈 128, 배치사이즈 100



## 0.36311- 887등

In [115]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_24 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 16, 16, 32)     

In [116]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-1model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [117]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [118]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("1model.h5")

Epoch 1/100
225/225 [==============================] - 199s 877ms/step - loss: 0.6137 - accuracy: 0.6527 - val_loss: 0.5551 - val_accuracy: 0.7288
Epoch 2/100
225/225 [==============================] - 186s 825ms/step - loss: 0.5541 - accuracy: 0.7148 - val_loss: 0.5145 - val_accuracy: 0.7460
Epoch 3/100
225/225 [==============================] - 183s 812ms/step - loss: 0.5232 - accuracy: 0.7394 - val_loss: 0.4813 - val_accuracy: 0.7572
Epoch 4/100
225/225 [==============================] - 185s 820ms/step - loss: 0.4878 - accuracy: 0.7668 - val_loss: 0.4943 - val_accuracy: 0.7600
Epoch 5/100
225/225 [==============================] - 192s 852ms/step - loss: 0.4726 - accuracy: 0.7781 - val_loss: 0.4528 - val_accuracy: 0.7832
Epoch 6/100
225/225 [==============================] - 184s 817ms/step - loss: 0.4637 - accuracy: 0.7866 - val_loss: 0.4291 - val_accuracy: 0.7936
Epoch 7/100
225/225 [==============================] - 180s 800ms/step - loss: 0.4474 - accuracy: 0.7980 - val_loss: 0

In [121]:
pred = model.predict(test_dataset)

125/125 [==============================] - 37s 295ms/step


In [127]:
save_preds(pred, "first_model.csv")

first_model.csv Saved!


In [119]:
model.evaluate_generator(test_dataset)

[1.4478065967559814, 0.5015199780464172]

In [120]:
model.evaluate(test_dataset)

125/125 [==============================] - 36s 289ms/step - loss: 1.4478 - accuracy: 0.5015


[1.4478065967559814, 0.5015199780464172]

# model(3) - model(2)의 conv2D(64), maxpool2d(2) 추가

## Score: 0.30503 - 859등

In [128]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 16, 16, 32)     

In [129]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-3model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [130]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [131]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("3model.h5")

Epoch 1/100
225/225 [==============================] - 201s 885ms/step - loss: 0.6478 - accuracy: 0.6076 - val_loss: 0.5838 - val_accuracy: 0.6764
Epoch 2/100
225/225 [==============================] - 186s 826ms/step - loss: 0.5646 - accuracy: 0.7076 - val_loss: 0.5241 - val_accuracy: 0.7272
Epoch 3/100
225/225 [==============================] - 185s 822ms/step - loss: 0.5202 - accuracy: 0.7446 - val_loss: 0.4835 - val_accuracy: 0.7792
Epoch 4/100
225/225 [==============================] - 185s 824ms/step - loss: 0.4873 - accuracy: 0.7692 - val_loss: 0.4540 - val_accuracy: 0.7804
Epoch 5/100
225/225 [==============================] - 183s 814ms/step - loss: 0.4716 - accuracy: 0.7815 - val_loss: 0.4525 - val_accuracy: 0.7968
Epoch 6/100
225/225 [==============================] - 186s 826ms/step - loss: 0.4450 - accuracy: 0.7982 - val_loss: 0.4028 - val_accuracy: 0.8128
Epoch 7/100
225/225 [==============================] - 189s 840ms/step - loss: 0.4263 - accuracy: 0.8103 - val_loss: 0

In [132]:
pred = model.predict(test_dataset)

125/125 [==============================] - 37s 291ms/step


In [133]:
save_preds(pred, "second_model.csv")

second_model.csv Saved!


# model(4)

- model(3)의 사이즈를 80으로 축소

## Score: 0.32353

In [136]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (80,80, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_29 (Conv2D)          (None, 80, 80, 8)         224       
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 40, 40, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 40, 40, 16)        1168      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 20, 20, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_31 (Conv2D)          (None, 20, 20, 32)        4640      
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 10, 10, 32)     

In [137]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-4model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [138]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [139]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("4model.h5")

Epoch 1/100
225/225 [==============================] - 120s 522ms/step - loss: 0.6370 - accuracy: 0.6335 - val_loss: 0.5885 - val_accuracy: 0.6856
Epoch 2/100
225/225 [==============================] - 118s 524ms/step - loss: 0.5753 - accuracy: 0.7006 - val_loss: 0.5464 - val_accuracy: 0.7212
Epoch 3/100
225/225 [==============================] - 116s 517ms/step - loss: 0.5470 - accuracy: 0.7256 - val_loss: 0.5324 - val_accuracy: 0.7232
Epoch 4/100
225/225 [==============================] - 115s 513ms/step - loss: 0.5135 - accuracy: 0.7524 - val_loss: 0.4785 - val_accuracy: 0.7676
Epoch 5/100
225/225 [==============================] - 116s 515ms/step - loss: 0.4935 - accuracy: 0.7656 - val_loss: 0.4567 - val_accuracy: 0.7836
Epoch 6/100
225/225 [==============================] - 116s 514ms/step - loss: 0.4697 - accuracy: 0.7833 - val_loss: 0.4358 - val_accuracy: 0.8008
Epoch 7/100
225/225 [==============================] - 116s 514ms/step - loss: 0.4515 - accuracy: 0.7947 - val_loss: 0

In [140]:
pred = model.predict(test_dataset)

125/125 [==============================] - 32s 253ms/step


In [141]:
save_preds(pred, "third_model.csv")

third_model.csv Saved!


# model(5)

- conv2d -> Dropout -> batchnorm -> acti -> pool 적용

## Score: 0.44185

In [142]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3,  padding = "same",
                              input_shape = (80,80, 3)))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3,  padding = "same"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3,  padding = "same"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3,  padding = "same"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 80, 80, 8)         224       
                                                                 
 dropout_13 (Dropout)        (None, 80, 80, 8)         0         
                                                                 
 batch_normalization (BatchN  (None, 80, 80, 8)        32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 80, 80, 8)         0         
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 40, 40, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 40, 40, 16)      

In [143]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-5model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [144]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [145]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("5model.h5")

Epoch 1/100
225/225 [==============================] - 131s 570ms/step - loss: 0.6555 - accuracy: 0.6061 - val_loss: 0.7492 - val_accuracy: 0.5000
Epoch 2/100
225/225 [==============================] - 128s 567ms/step - loss: 0.6028 - accuracy: 0.6721 - val_loss: 0.8097 - val_accuracy: 0.5008
Epoch 3/100
225/225 [==============================] - 127s 566ms/step - loss: 0.5676 - accuracy: 0.7048 - val_loss: 0.7531 - val_accuracy: 0.5212
Epoch 4/100
225/225 [==============================] - 128s 567ms/step - loss: 0.5360 - accuracy: 0.7329 - val_loss: 0.6136 - val_accuracy: 0.6448
Epoch 5/100
225/225 [==============================] - 127s 566ms/step - loss: 0.5104 - accuracy: 0.7503 - val_loss: 0.5656 - val_accuracy: 0.7044
Epoch 6/100
225/225 [==============================] - 128s 566ms/step - loss: 0.4975 - accuracy: 0.7596 - val_loss: 0.5224 - val_accuracy: 0.7796
Epoch 7/100
225/225 [==============================] - 127s 563ms/step - loss: 0.4824 - accuracy: 0.7717 - val_loss: 0

In [146]:
pred = model.predict(test_dataset)

125/125 [==============================] - 33s 264ms/step


In [147]:
save_preds(pred, "4_model.csv")

4_model.csv Saved!


# model(6)

## Score: 0.47804

In [148]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3,  padding = "same",
                              input_shape = (80,80, 3)))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3,  padding = "same"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3,  padding = "same"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3,  padding = "same"))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(2))


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_37 (Conv2D)          (None, 80, 80, 8)         224       
                                                                 
 dropout_18 (Dropout)        (None, 80, 80, 8)         0         
                                                                 
 batch_normalization_4 (Batc  (None, 80, 80, 8)        32        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 80, 80, 8)         0         
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 40, 40, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 40, 40, 16)      

In [149]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-6model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [150]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [151]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("6model.h5")

Epoch 1/100
225/225 [==============================] - 130s 568ms/step - loss: 0.6777 - accuracy: 0.5754 - val_loss: 0.6971 - val_accuracy: 0.5000
Epoch 2/100
225/225 [==============================] - 127s 563ms/step - loss: 0.6215 - accuracy: 0.6706 - val_loss: 1.0180 - val_accuracy: 0.5004
Epoch 3/100
225/225 [==============================] - 126s 559ms/step - loss: 0.5762 - accuracy: 0.7064 - val_loss: 0.9160 - val_accuracy: 0.5124
Epoch 4/100
225/225 [==============================] - 127s 564ms/step - loss: 0.5348 - accuracy: 0.7376 - val_loss: 0.6933 - val_accuracy: 0.6180
Epoch 5/100
225/225 [==============================] - 127s 562ms/step - loss: 0.5115 - accuracy: 0.7543 - val_loss: 0.5742 - val_accuracy: 0.7240
Epoch 6/100
225/225 [==============================] - 127s 563ms/step - loss: 0.4874 - accuracy: 0.7715 - val_loss: 0.6829 - val_accuracy: 0.5968
Epoch 7/100
225/225 [==============================] - 126s 561ms/step - loss: 0.4715 - accuracy: 0.7805 - val_loss: 0

In [152]:
pred = model.predict(test_dataset)

125/125 [==============================] - 33s 263ms/step


In [153]:
save_preds(pred, "5_model.csv")

5_model.csv Saved!


# model(7) - model(3)의 사이즈 128이 가장 좋은 모델

## Score: 0.26587  - 836등

In [156]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_46 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_46 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_47 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_48 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 16, 16, 32)     

In [157]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-7model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [158]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [159]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("7model.h5")

Epoch 1/100
225/225 [==============================] - 187s 825ms/step - loss: 0.6488 - accuracy: 0.6126 - val_loss: 0.5702 - val_accuracy: 0.7096
Epoch 2/100
225/225 [==============================] - 185s 822ms/step - loss: 0.5608 - accuracy: 0.7178 - val_loss: 0.5053 - val_accuracy: 0.7496
Epoch 3/100
225/225 [==============================] - 185s 820ms/step - loss: 0.5008 - accuracy: 0.7596 - val_loss: 0.4578 - val_accuracy: 0.7804
Epoch 4/100
225/225 [==============================] - 184s 819ms/step - loss: 0.4550 - accuracy: 0.7929 - val_loss: 0.3913 - val_accuracy: 0.8224
Epoch 5/100
225/225 [==============================] - 185s 823ms/step - loss: 0.4206 - accuracy: 0.8130 - val_loss: 0.3757 - val_accuracy: 0.8220
Epoch 6/100
225/225 [==============================] - 186s 825ms/step - loss: 0.3945 - accuracy: 0.8255 - val_loss: 0.3390 - val_accuracy: 0.8532
Epoch 7/100
225/225 [==============================] - 186s 826ms/step - loss: 0.3618 - accuracy: 0.8455 - val_loss: 0

In [160]:
pred = model.predict(test_dataset)

125/125 [==============================] - 37s 291ms/step


In [161]:
save_preds(pred, "6_model.csv")

6_model.csv Saved!


# model(8)

- model(7)에 dense 16 추가

## Score: 0.33196

In [163]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_58 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_59 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_60 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 16, 16, 32)     

In [164]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-8model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [165]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [166]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("8model.h5")

Epoch 1/100
225/225 [==============================] - 191s 834ms/step - loss: 0.6698 - accuracy: 0.5770 - val_loss: 0.6776 - val_accuracy: 0.5804
Epoch 2/100
225/225 [==============================] - 186s 827ms/step - loss: 0.5946 - accuracy: 0.6924 - val_loss: 0.5095 - val_accuracy: 0.7552
Epoch 3/100
225/225 [==============================] - 185s 823ms/step - loss: 0.5180 - accuracy: 0.7530 - val_loss: 0.4685 - val_accuracy: 0.7692
Epoch 4/100
225/225 [==============================] - 188s 835ms/step - loss: 0.4618 - accuracy: 0.7931 - val_loss: 0.4115 - val_accuracy: 0.8168
Epoch 5/100
225/225 [==============================] - 186s 827ms/step - loss: 0.4240 - accuracy: 0.8150 - val_loss: 0.4104 - val_accuracy: 0.8184
Epoch 6/100
225/225 [==============================] - 185s 824ms/step - loss: 0.3989 - accuracy: 0.8287 - val_loss: 0.3915 - val_accuracy: 0.8224
Epoch 7/100
225/225 [==============================] - 186s 825ms/step - loss: 0.3747 - accuracy: 0.8426 - val_loss: 0

In [167]:
pred = model.predict(test_dataset)

125/125 [==============================] - 36s 290ms/step


In [168]:
save_preds(pred, "7_model.csv")

7_model.csv Saved!


# model(9) - model(7)에 dense 16 상위에 추가

## Score: 0.26122 - 833등

In [169]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_63 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_63 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_64 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_64 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_65 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_65 (MaxPoolin  (None, 16, 16, 32)     

In [170]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-9model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [171]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [172]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("9model.h5")

Epoch 1/100
225/225 [==============================] - 188s 827ms/step - loss: 0.6731 - accuracy: 0.5819 - val_loss: 0.6335 - val_accuracy: 0.6736
Epoch 2/100
225/225 [==============================] - 186s 824ms/step - loss: 0.6035 - accuracy: 0.6884 - val_loss: 0.5943 - val_accuracy: 0.6816
Epoch 3/100
225/225 [==============================] - 185s 824ms/step - loss: 0.5313 - accuracy: 0.7488 - val_loss: 0.4803 - val_accuracy: 0.7688
Epoch 4/100
225/225 [==============================] - 186s 826ms/step - loss: 0.4759 - accuracy: 0.7834 - val_loss: 0.4551 - val_accuracy: 0.7832
Epoch 5/100
225/225 [==============================] - 186s 824ms/step - loss: 0.4347 - accuracy: 0.8065 - val_loss: 0.4154 - val_accuracy: 0.8156
Epoch 6/100
225/225 [==============================] - 185s 824ms/step - loss: 0.4044 - accuracy: 0.8237 - val_loss: 0.3889 - val_accuracy: 0.8272
Epoch 7/100
225/225 [==============================] - 186s 825ms/step - loss: 0.3687 - accuracy: 0.8426 - val_loss: 0

In [173]:
pred = model.predict(test_dataset)

125/125 [==============================] - 39s 306ms/step


In [174]:
save_preds(pred, "8_model.csv")

8_model.csv Saved!


# model(10)

- model(8)에 dense 8 추가

## Score: 0.28973

In [175]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(8, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_68 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_68 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_69 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_69 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_70 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_70 (MaxPoolin  (None, 16, 16, 32)     

In [176]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-10model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [177]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [178]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("10model.h5")

Epoch 1/100
225/225 [==============================] - 202s 887ms/step - loss: 0.6816 - accuracy: 0.5594 - val_loss: 0.6498 - val_accuracy: 0.6200
Epoch 2/100
225/225 [==============================] - 199s 884ms/step - loss: 0.6431 - accuracy: 0.6469 - val_loss: 0.5867 - val_accuracy: 0.6904
Epoch 3/100
225/225 [==============================] - 199s 883ms/step - loss: 0.5889 - accuracy: 0.7074 - val_loss: 0.5514 - val_accuracy: 0.7296
Epoch 4/100
225/225 [==============================] - 199s 884ms/step - loss: 0.5403 - accuracy: 0.7461 - val_loss: 0.4835 - val_accuracy: 0.7696
Epoch 5/100
225/225 [==============================] - 199s 882ms/step - loss: 0.5051 - accuracy: 0.7737 - val_loss: 0.4597 - val_accuracy: 0.7796
Epoch 6/100
225/225 [==============================] - 199s 882ms/step - loss: 0.4605 - accuracy: 0.8012 - val_loss: 0.4087 - val_accuracy: 0.8088
Epoch 7/100
225/225 [==============================] - 199s 884ms/step - loss: 0.4314 - accuracy: 0.8197 - val_loss: 0

In [179]:
pred = model.predict(test_dataset)

125/125 [==============================] - 47s 376ms/step


In [180]:
save_preds(pred, "9_model.csv")

9_model.csv Saved!


# model(11)

- model(9)에 dense 8 위로 추가

## Score: 0.26639

In [181]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(8, activation = "relu"))

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_73 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_73 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_74 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_74 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_75 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 16, 16, 32)     

In [182]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-11model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [183]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [184]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("11model.h5")

Epoch 1/100
225/225 [==============================] - 201s 885ms/step - loss: 0.6618 - accuracy: 0.5991 - val_loss: 0.6335 - val_accuracy: 0.6444
Epoch 2/100
225/225 [==============================] - 199s 882ms/step - loss: 0.5909 - accuracy: 0.6973 - val_loss: 0.5352 - val_accuracy: 0.7400
Epoch 3/100
225/225 [==============================] - 199s 885ms/step - loss: 0.5227 - accuracy: 0.7542 - val_loss: 0.4951 - val_accuracy: 0.7596
Epoch 4/100
225/225 [==============================] - 199s 884ms/step - loss: 0.4682 - accuracy: 0.7879 - val_loss: 0.4195 - val_accuracy: 0.8088
Epoch 5/100
225/225 [==============================] - 199s 884ms/step - loss: 0.4400 - accuracy: 0.8080 - val_loss: 0.4066 - val_accuracy: 0.8192
Epoch 6/100
225/225 [==============================] - 199s 886ms/step - loss: 0.4122 - accuracy: 0.8215 - val_loss: 0.3797 - val_accuracy: 0.8340
Epoch 7/100
225/225 [==============================] - 199s 885ms/step - loss: 0.3792 - accuracy: 0.8366 - val_loss: 0

In [185]:
pred = model.predict(test_dataset)

125/125 [==============================] - 38s 305ms/step


In [186]:
save_preds(pred, "10_model.csv")

10_model.csv Saved!


# model(12)

- model(9)에 dense 16을 빼고 64 추가

## Score: 0.27733

In [187]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dense(64, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_78 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_78 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_79 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_79 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_80 (Conv2D)          (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_80 (MaxPoolin  (None, 16, 16, 32)     

In [188]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-12model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [189]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [190]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("12model.h5")

Epoch 1/100
225/225 [==============================] - 207s 908ms/step - loss: 0.6660 - accuracy: 0.5872 - val_loss: 0.6074 - val_accuracy: 0.6856
Epoch 2/100
225/225 [==============================] - 197s 877ms/step - loss: 0.5686 - accuracy: 0.7104 - val_loss: 0.5055 - val_accuracy: 0.7612
Epoch 3/100
225/225 [==============================] - 194s 861ms/step - loss: 0.4954 - accuracy: 0.7659 - val_loss: 0.4449 - val_accuracy: 0.7936
Epoch 4/100
225/225 [==============================] - 196s 873ms/step - loss: 0.4464 - accuracy: 0.7948 - val_loss: 0.3987 - val_accuracy: 0.8180
Epoch 5/100
225/225 [==============================] - 192s 854ms/step - loss: 0.3989 - accuracy: 0.8217 - val_loss: 0.3593 - val_accuracy: 0.8388
Epoch 6/100
225/225 [==============================] - 202s 896ms/step - loss: 0.3734 - accuracy: 0.8359 - val_loss: 0.3742 - val_accuracy: 0.8432
Epoch 7/100
225/225 [==============================] - 188s 834ms/step - loss: 0.3428 - accuracy: 0.8516 - val_loss: 0

In [191]:
pred = model.predict(test_dataset)

125/125 [==============================] - 48s 382ms/step


In [192]:
save_preds(pred, "11_model.csv")

11_model.csv Saved!


# model(13)

- model(9)에 dropout 추가

## Score: 0.44345

In [193]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_83 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_83 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 dropout_31 (Dropout)        (None, 64, 64, 8)         0         
                                                                 
 conv2d_84 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_84 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_85 (Conv2D)          (None, 32, 32, 32)      

In [194]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-13model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [195]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [196]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("13model.h5")

Epoch 1/100
225/225 [==============================] - 210s 924ms/step - loss: 0.6845 - accuracy: 0.5338 - val_loss: 0.6676 - val_accuracy: 0.5752
Epoch 2/100
225/225 [==============================] - 201s 891ms/step - loss: 0.6165 - accuracy: 0.6575 - val_loss: 0.5767 - val_accuracy: 0.7160
Epoch 3/100
225/225 [==============================] - 202s 899ms/step - loss: 0.5570 - accuracy: 0.7176 - val_loss: 0.5659 - val_accuracy: 0.7032
Epoch 4/100
225/225 [==============================] - 190s 843ms/step - loss: 0.5078 - accuracy: 0.7527 - val_loss: 0.5032 - val_accuracy: 0.7540
Epoch 5/100
225/225 [==============================] - 188s 837ms/step - loss: 0.4683 - accuracy: 0.7777 - val_loss: 0.5605 - val_accuracy: 0.6984
Epoch 6/100
225/225 [==============================] - 188s 835ms/step - loss: 0.4442 - accuracy: 0.7916 - val_loss: 0.4451 - val_accuracy: 0.7828
Epoch 7/100
225/225 [==============================] - 190s 843ms/step - loss: 0.4225 - accuracy: 0.8077 - val_loss: 0

In [197]:
pred = model.predict(test_dataset)

125/125 [==============================] - 48s 382ms/step


In [198]:
save_preds(pred, "12_model.csv")

12_model.csv Saved!


# model(14)

- model(12)에 dropout 추가

## Score: 0.46512

In [199]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dense(64, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_88 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_88 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 dropout_33 (Dropout)        (None, 64, 64, 8)         0         
                                                                 
 conv2d_89 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_89 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_90 (Conv2D)          (None, 32, 32, 32)      

In [200]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-14model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [201]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [202]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("14model.h5")

Epoch 1/100
225/225 [==============================] - 190s 835ms/step - loss: 0.6484 - accuracy: 0.6071 - val_loss: 0.6072 - val_accuracy: 0.6704
Epoch 2/100
225/225 [==============================] - 198s 881ms/step - loss: 0.5626 - accuracy: 0.7093 - val_loss: 0.6119 - val_accuracy: 0.6440
Epoch 3/100
225/225 [==============================] - 191s 849ms/step - loss: 0.5070 - accuracy: 0.7530 - val_loss: 0.6003 - val_accuracy: 0.6856
Epoch 4/100
225/225 [==============================] - 189s 839ms/step - loss: 0.4717 - accuracy: 0.7787 - val_loss: 0.4995 - val_accuracy: 0.7400
Epoch 5/100
225/225 [==============================] - 188s 836ms/step - loss: 0.4418 - accuracy: 0.7949 - val_loss: 0.5565 - val_accuracy: 0.7232
Epoch 6/100
225/225 [==============================] - 188s 836ms/step - loss: 0.4130 - accuracy: 0.8145 - val_loss: 0.5028 - val_accuracy: 0.7420
Epoch 7/100
225/225 [==============================] - 188s 837ms/step - loss: 0.3923 - accuracy: 0.8228 - val_loss: 0

In [203]:
pred = model.predict(test_dataset)

125/125 [==============================] - 49s 388ms/step


In [204]:
save_preds(pred, "13_model.csv")

13_model.csv Saved!


# model(15)

- model(9)의 activate = "leakyrelu"로 변경

## Score: 0.27665

In [207]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_98 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_98 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_99 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_99 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_100 (Conv2D)         (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_100 (MaxPooli  (None, 16, 16, 32)     

In [208]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-15model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [209]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [210]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("15model.h5")

Epoch 1/100
225/225 [==============================] - 191s 842ms/step - loss: 0.6585 - accuracy: 0.5952 - val_loss: 0.5898 - val_accuracy: 0.6776
Epoch 2/100
225/225 [==============================] - 188s 836ms/step - loss: 0.5614 - accuracy: 0.7172 - val_loss: 0.5119 - val_accuracy: 0.7480
Epoch 3/100
225/225 [==============================] - 188s 836ms/step - loss: 0.4900 - accuracy: 0.7691 - val_loss: 0.4699 - val_accuracy: 0.7724
Epoch 4/100
225/225 [==============================] - 188s 833ms/step - loss: 0.4398 - accuracy: 0.8028 - val_loss: 0.4452 - val_accuracy: 0.7832
Epoch 5/100
225/225 [==============================] - 188s 834ms/step - loss: 0.4004 - accuracy: 0.8234 - val_loss: 0.3677 - val_accuracy: 0.8324
Epoch 6/100
225/225 [==============================] - 189s 842ms/step - loss: 0.3731 - accuracy: 0.8379 - val_loss: 0.3620 - val_accuracy: 0.8408
Epoch 7/100
225/225 [==============================] - 207s 921ms/step - loss: 0.3404 - accuracy: 0.8560 - val_loss: 0

In [211]:
pred = model.predict(test_dataset)

125/125 [==============================] - 49s 387ms/step


In [212]:
save_preds(pred, "14_model.csv")

14_model.csv Saved!


# model(16)

- leaky_relu 2개 추가

## Score: 0.28635

In [213]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_103 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 max_pooling2d_103 (MaxPooli  (None, 64, 64, 8)        0         
 ng2D)                                                           
                                                                 
 conv2d_104 (Conv2D)         (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_104 (MaxPooli  (None, 32, 32, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_105 (Conv2D)         (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_105 (MaxPooli  (None, 16, 16, 32)     

In [214]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-16model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [215]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [216]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("16model.h5")

Epoch 1/100
225/225 [==============================] - 228s 1s/step - loss: 0.6848 - accuracy: 0.5372 - val_loss: 0.6575 - val_accuracy: 0.6272
Epoch 2/100
225/225 [==============================] - 216s 958ms/step - loss: 0.6130 - accuracy: 0.6761 - val_loss: 0.5727 - val_accuracy: 0.7196
Epoch 3/100
225/225 [==============================] - 213s 948ms/step - loss: 0.5512 - accuracy: 0.7328 - val_loss: 0.4842 - val_accuracy: 0.7780
Epoch 4/100
225/225 [==============================] - 213s 945ms/step - loss: 0.4940 - accuracy: 0.7752 - val_loss: 0.4535 - val_accuracy: 0.8008
Epoch 5/100
225/225 [==============================] - 213s 945ms/step - loss: 0.4463 - accuracy: 0.8010 - val_loss: 0.4275 - val_accuracy: 0.8012
Epoch 6/100
225/225 [==============================] - 212s 941ms/step - loss: 0.4110 - accuracy: 0.8218 - val_loss: 0.3593 - val_accuracy: 0.8440
Epoch 7/100
225/225 [==============================] - 213s 944ms/step - loss: 0.3868 - accuracy: 0.8356 - val_loss: 0.34

In [217]:
pred = model.predict(test_dataset)

125/125 [==============================] - 48s 378ms/step


In [218]:
save_preds(pred, "15_model.csv")

15_model.csv Saved!


# model(17) - model(9)에서 첫번째 maxpool을 제거

## Score: 0.24516 - 821등

In [220]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_113 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_114 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_112 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_115 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_113 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_116 (Conv2D)         (None, 32, 32, 64)      

In [221]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-17model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [222]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [223]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb])
model.save_weights("17model.h5")

Epoch 1/100
225/225 [==============================] - 255s 1s/step - loss: 0.6669 - accuracy: 0.5753 - val_loss: 0.6183 - val_accuracy: 0.6536
Epoch 2/100
225/225 [==============================] - 241s 1s/step - loss: 0.5649 - accuracy: 0.7063 - val_loss: 0.5162 - val_accuracy: 0.7436
Epoch 3/100
225/225 [==============================] - 256s 1s/step - loss: 0.5051 - accuracy: 0.7588 - val_loss: 0.4709 - val_accuracy: 0.7768
Epoch 4/100
225/225 [==============================] - 265s 1s/step - loss: 0.4597 - accuracy: 0.7868 - val_loss: 0.4561 - val_accuracy: 0.7752
Epoch 5/100
225/225 [==============================] - 272s 1s/step - loss: 0.4109 - accuracy: 0.8167 - val_loss: 0.4625 - val_accuracy: 0.7816
Epoch 6/100
225/225 [==============================] - 284s 1s/step - loss: 0.3866 - accuracy: 0.8308 - val_loss: 0.3634 - val_accuracy: 0.8356
Epoch 7/100
225/225 [==============================] - 280s 1s/step - loss: 0.3485 - accuracy: 0.8511 - val_loss: 0.3618 - val_accuracy:

In [224]:
pred = model.predict(test_dataset)

125/125 [==============================] - 53s 426ms/step


In [225]:
save_preds(pred, "16_model.csv")

16_model.csv Saved!


# model(18)

- model(17)에 reduce를 걸어봄

## Score: 0.28464

In [230]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_123 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_124 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_120 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_125 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_121 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_126 (Conv2D)         (None, 32, 32, 64)      

In [231]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-18model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [232]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [233]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("18model.h5")

Epoch 1/100
225/225 [==============================] - 260s 1s/step - loss: 0.6767 - accuracy: 0.5736 - val_loss: 0.6277 - val_accuracy: 0.6700 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 253s 1s/step - loss: 0.5983 - accuracy: 0.6965 - val_loss: 0.5690 - val_accuracy: 0.7164 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 253s 1s/step - loss: 0.5389 - accuracy: 0.7408 - val_loss: 0.4806 - val_accuracy: 0.7788 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 254s 1s/step - loss: 0.4953 - accuracy: 0.7690 - val_loss: 0.4411 - val_accuracy: 0.7956 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 253s 1s/step - loss: 0.4444 - accuracy: 0.8000 - val_loss: 0.4114 - val_accuracy: 0.8120 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 250s 1s/step - loss: 0.4083 - accuracy: 0.8211 - val_loss: 0.3676 - val_accuracy: 0.8392 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 252

In [234]:
pred = model.predict(test_dataset)

125/125 [==============================] - 53s 422ms/step


In [235]:
save_preds(pred, "17_model.csv")

17_model.csv Saved!


# model(19) -dropout(0.25) 추가

## Score: 0.27115

In [236]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_128 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 dropout_42 (Dropout)        (None, 128, 128, 8)       0         
                                                                 
 conv2d_129 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_124 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_130 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_125 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                               

In [237]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-19model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [238]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [239]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("19model.h5")

Epoch 1/100
225/225 [==============================] - 277s 1s/step - loss: 0.6843 - accuracy: 0.5460 - val_loss: 0.7119 - val_accuracy: 0.5244 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 265s 1s/step - loss: 0.6082 - accuracy: 0.6787 - val_loss: 0.5918 - val_accuracy: 0.6756 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 265s 1s/step - loss: 0.5538 - accuracy: 0.7223 - val_loss: 0.5151 - val_accuracy: 0.7432 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 266s 1s/step - loss: 0.4960 - accuracy: 0.7654 - val_loss: 0.4596 - val_accuracy: 0.7812 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 266s 1s/step - loss: 0.4590 - accuracy: 0.7877 - val_loss: 0.4259 - val_accuracy: 0.8012 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 266s 1s/step - loss: 0.4216 - accuracy: 0.8085 - val_loss: 0.3906 - val_accuracy: 0.8232 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 266

In [240]:
pred = model.predict(test_dataset)

125/125 [==============================] - 53s 425ms/step


In [241]:
save_preds(pred, "18_model.csv")

18_model.csv Saved!


# model(20) - dropout(0.25)추가

## Score: 0.24134 - 821등

In [246]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_138 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_139 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_132 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 dropout_46 (Dropout)        (None, 64, 64, 16)        0         
                                                                 
 conv2d_140 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_133 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                               

In [247]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-20model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [248]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [249]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("20model.h5")

Epoch 1/100
225/225 [==============================] - 263s 1s/step - loss: 0.6787 - accuracy: 0.5692 - val_loss: 0.6520 - val_accuracy: 0.6280 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 248s 1s/step - loss: 0.5937 - accuracy: 0.6884 - val_loss: 0.5469 - val_accuracy: 0.7352 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 245s 1s/step - loss: 0.5172 - accuracy: 0.7533 - val_loss: 0.4686 - val_accuracy: 0.7732 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 247s 1s/step - loss: 0.4595 - accuracy: 0.7879 - val_loss: 0.4079 - val_accuracy: 0.8136 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 245s 1s/step - loss: 0.4221 - accuracy: 0.8096 - val_loss: 0.4151 - val_accuracy: 0.8120 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 244s 1s/step - loss: 0.3877 - accuracy: 0.8291 - val_loss: 0.4020 - val_accuracy: 0.8192 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 246

In [250]:
pred = model.predict(test_dataset)

125/125 [==============================] - 55s 441ms/step


In [251]:
save_preds(pred, "19_model.csv")

19_model.csv Saved!


# model(21) - dropout(0.25) 추가

## Score: 0.32030

In [252]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_143 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_144 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_136 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_145 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_137 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                                           
                                                                 
 dropout_48 (Dropout)        (None, 32, 32, 32)      

In [253]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-21model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [254]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [255]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("21model.h5")

Epoch 1/100
225/225 [==============================] - 324s 1s/step - loss: 0.6936 - accuracy: 0.4989 - val_loss: 0.6927 - val_accuracy: 0.5080 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 320s 1s/step - loss: 0.6831 - accuracy: 0.5615 - val_loss: 0.6774 - val_accuracy: 0.5660 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 319s 1s/step - loss: 0.6353 - accuracy: 0.6313 - val_loss: 0.5497 - val_accuracy: 0.7144 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 321s 1s/step - loss: 0.5591 - accuracy: 0.7136 - val_loss: 0.5055 - val_accuracy: 0.7564 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 321s 1s/step - loss: 0.5161 - accuracy: 0.7479 - val_loss: 0.4689 - val_accuracy: 0.7772 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 321s 1s/step - loss: 0.4884 - accuracy: 0.7696 - val_loss: 0.4460 - val_accuracy: 0.7928 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 320

In [256]:
pred = model.predict(test_dataset)

125/125 [==============================] - 56s 443ms/step


In [257]:
save_preds(pred, "20_model.csv")

20_model.csv Saved!


# model(22) - dropout(0.25) 추가

## Score: 0.22391 - 807등

In [258]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_148 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_149 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_140 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_150 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_141 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_151 (Conv2D)         (None, 32, 32, 64)      

In [259]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-22model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [260]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [261]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("22model.h5")

Epoch 1/100
225/225 [==============================] - 327s 1s/step - loss: 0.6752 - accuracy: 0.5662 - val_loss: 0.6386 - val_accuracy: 0.6096 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 323s 1s/step - loss: 0.5969 - accuracy: 0.6888 - val_loss: 0.5092 - val_accuracy: 0.7452 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 324s 1s/step - loss: 0.4992 - accuracy: 0.7624 - val_loss: 0.4693 - val_accuracy: 0.7776 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 324s 1s/step - loss: 0.4491 - accuracy: 0.7957 - val_loss: 0.4109 - val_accuracy: 0.8160 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 323s 1s/step - loss: 0.4105 - accuracy: 0.8168 - val_loss: 0.3782 - val_accuracy: 0.8392 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 321s 1s/step - loss: 0.3753 - accuracy: 0.8353 - val_loss: 0.3689 - val_accuracy: 0.8228 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 325

In [262]:
pred = model.predict(test_dataset)

125/125 [==============================] - 56s 445ms/step


In [263]:
save_preds(pred, "21_model.csv")

21_model.csv Saved!


# model(23) - dropout(0.25) 추가

## Score: 0.23165

In [264]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "relu"))

model.add(keras.layers.Dense(32, activation = "relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_153 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_154 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_144 (MaxPooli  (None, 64, 64, 16)       0         
 ng2D)                                                           
                                                                 
 conv2d_155 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_145 (MaxPooli  (None, 32, 32, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_156 (Conv2D)         (None, 32, 32, 64)      

In [265]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-23model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [266]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [267]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("23model.h5")

Epoch 1/100
225/225 [==============================] - 327s 1s/step - loss: 0.6663 - accuracy: 0.5785 - val_loss: 0.5962 - val_accuracy: 0.6616 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 323s 1s/step - loss: 0.5742 - accuracy: 0.6966 - val_loss: 0.5385 - val_accuracy: 0.7228 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 323s 1s/step - loss: 0.5172 - accuracy: 0.7448 - val_loss: 0.4825 - val_accuracy: 0.7580 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 322s 1s/step - loss: 0.4614 - accuracy: 0.7880 - val_loss: 0.4041 - val_accuracy: 0.8132 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 321s 1s/step - loss: 0.4279 - accuracy: 0.8075 - val_loss: 0.3990 - val_accuracy: 0.8188 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 321s 1s/step - loss: 0.3973 - accuracy: 0.8217 - val_loss: 0.3815 - val_accuracy: 0.8236 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 323

In [268]:
pred = model.predict(test_dataset)

125/125 [==============================] - 52s 414ms/step


In [269]:
save_preds(pred, "22_model.csv")

22_model.csv Saved!


# model(24) - model(22)에 dense층에 leaky_relu로 변경

## Score: 0.24751

In [11]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 8)       224       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 64)        1

2023-02-09 11:14:03.489431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-224model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [13]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [14]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("24model.h5")

Epoch 1/100
225/225 [==============================] - 259s 1s/step - loss: 0.6549 - accuracy: 0.5933 - val_loss: 0.5954 - val_accuracy: 0.6796 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 249s 1s/step - loss: 0.5620 - accuracy: 0.7085 - val_loss: 0.5062 - val_accuracy: 0.7396 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 250s 1s/step - loss: 0.5033 - accuracy: 0.7533 - val_loss: 0.4709 - val_accuracy: 0.7740 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 250s 1s/step - loss: 0.4570 - accuracy: 0.7868 - val_loss: 0.4292 - val_accuracy: 0.8084 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 251s 1s/step - loss: 0.4076 - accuracy: 0.8150 - val_loss: 0.4459 - val_accuracy: 0.7848 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 251s 1s/step - loss: 0.3725 - accuracy: 0.8356 - val_loss: 0.3708 - val_accuracy: 0.8356 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 273

In [15]:
pred = model.predict(test_dataset)

125/125 [==============================] - 47s 376ms/step


In [16]:
save_preds(pred, "23_model.csv")

23_model.csv Saved!


# model(25) - model(24)에 leaky_relu 추가

## Score: 0.22939

In [17]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 128, 128, 8)       224       
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 16)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 32, 32, 64)       

In [18]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-25model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [19]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [20]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("25model.h5")

Epoch 1/100
225/225 [==============================] - 257s 1s/step - loss: 0.6314 - accuracy: 0.6268 - val_loss: 0.5347 - val_accuracy: 0.7300 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 256s 1s/step - loss: 0.5428 - accuracy: 0.7283 - val_loss: 0.4988 - val_accuracy: 0.7620 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 259s 1s/step - loss: 0.4826 - accuracy: 0.7730 - val_loss: 0.4342 - val_accuracy: 0.7932 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 259s 1s/step - loss: 0.4268 - accuracy: 0.8033 - val_loss: 0.4014 - val_accuracy: 0.8212 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 258s 1s/step - loss: 0.3941 - accuracy: 0.8240 - val_loss: 0.3581 - val_accuracy: 0.8372 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 259s 1s/step - loss: 0.3582 - accuracy: 0.8435 - val_loss: 0.3308 - val_accuracy: 0.8504 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 259

In [21]:
pred = model.predict(test_dataset)

125/125 [==============================] - 48s 379ms/step


In [22]:
save_preds(pred, "24_model.csv")

24_model.csv Saved!


# model(26) - model(25)에 leaky_relu 하나 더 추가

## Score: 0.22229 - 804등

In [27]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_16 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 32, 32, 64)       

In [28]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-26model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [29]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [30]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("26model.h5")

Epoch 1/100
225/225 [==============================] - 278s 1s/step - loss: 0.6767 - accuracy: 0.5483 - val_loss: 0.6160 - val_accuracy: 0.6468 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 273s 1s/step - loss: 0.5667 - accuracy: 0.7025 - val_loss: 0.5034 - val_accuracy: 0.7468 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 270s 1s/step - loss: 0.4992 - accuracy: 0.7552 - val_loss: 0.4544 - val_accuracy: 0.7836 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 272s 1s/step - loss: 0.4378 - accuracy: 0.7972 - val_loss: 0.4023 - val_accuracy: 0.8204 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 272s 1s/step - loss: 0.3943 - accuracy: 0.8229 - val_loss: 0.3785 - val_accuracy: 0.8276 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 271s 1s/step - loss: 0.3550 - accuracy: 0.8430 - val_loss: 0.3409 - val_accuracy: 0.8460 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 273

In [31]:
pred = model.predict(test_dataset)

125/125 [==============================] - 50s 400ms/step


In [32]:
save_preds(pred, "25_model.csv")

25_model.csv Saved!


# model(27) - model(25)에 leaky_relu 자리 바꿔서 추가

## Score: 0.21764 - 799등

In [33]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_21 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_23 (Conv2D)          (None, 32, 32, 64)       

In [34]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-27model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [35]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [36]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("27model.h5")

Epoch 1/100
225/225 [==============================] - 301s 1s/step - loss: 0.6780 - accuracy: 0.5586 - val_loss: 0.6095 - val_accuracy: 0.6560 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 277s 1s/step - loss: 0.5722 - accuracy: 0.7054 - val_loss: 0.5089 - val_accuracy: 0.7508 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 266s 1s/step - loss: 0.4792 - accuracy: 0.7748 - val_loss: 0.4325 - val_accuracy: 0.7960 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 269s 1s/step - loss: 0.4296 - accuracy: 0.8058 - val_loss: 0.3953 - val_accuracy: 0.8100 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 291s 1s/step - loss: 0.3874 - accuracy: 0.8298 - val_loss: 0.3561 - val_accuracy: 0.8460 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 274s 1s/step - loss: 0.3539 - accuracy: 0.8449 - val_loss: 0.3400 - val_accuracy: 0.8516 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 275

In [37]:
pred = model.predict(test_dataset)

125/125 [==============================] - 48s 380ms/step


In [38]:
save_preds(pred, "26_model.csv")

26_model.csv Saved!


# model(28) - model(27)에 dropout(0.25)를 Flatten 밑에 추가

## Score: 0.19314 - 776등

In [39]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_25 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_26 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_27 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 32, 32, 64)       

In [40]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-28model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [41]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [42]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("28model.h5")

Epoch 1/100
225/225 [==============================] - 250s 1s/step - loss: 0.6734 - accuracy: 0.5680 - val_loss: 0.6091 - val_accuracy: 0.6780 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 232s 1s/step - loss: 0.5841 - accuracy: 0.6955 - val_loss: 0.5288 - val_accuracy: 0.7328 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 232s 1s/step - loss: 0.4974 - accuracy: 0.7576 - val_loss: 0.4565 - val_accuracy: 0.7768 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 232s 1s/step - loss: 0.4297 - accuracy: 0.8047 - val_loss: 0.4430 - val_accuracy: 0.7864 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 232s 1s/step - loss: 0.3917 - accuracy: 0.8264 - val_loss: 0.3520 - val_accuracy: 0.8420 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 233s 1s/step - loss: 0.3504 - accuracy: 0.8474 - val_loss: 0.3368 - val_accuracy: 0.8552 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 233

In [43]:
pred = model.predict(test_dataset)

125/125 [==============================] - 48s 383ms/step


In [44]:
save_preds(pred, "27_model.csv")

27_model.csv Saved!


# model(29)- model(27)에 dropout(0.25) 위치 바꿔서 추가

## Score: 0.21283

In [45]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_31 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_32 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 32, 32, 64)       

In [46]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-29model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [47]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [48]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("29model.h5")

Epoch 1/100
225/225 [==============================] - 263s 1s/step - loss: 0.6396 - accuracy: 0.6238 - val_loss: 0.5440 - val_accuracy: 0.7120 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 261s 1s/step - loss: 0.5145 - accuracy: 0.7481 - val_loss: 0.4535 - val_accuracy: 0.7836 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 261s 1s/step - loss: 0.4522 - accuracy: 0.7922 - val_loss: 0.4056 - val_accuracy: 0.8188 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 261s 1s/step - loss: 0.4056 - accuracy: 0.8157 - val_loss: 0.3561 - val_accuracy: 0.8416 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 262s 1s/step - loss: 0.3675 - accuracy: 0.8371 - val_loss: 0.3405 - val_accuracy: 0.8484 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 261s 1s/step - loss: 0.3315 - accuracy: 0.8537 - val_loss: 0.3060 - val_accuracy: 0.8712 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 262

In [49]:
pred = model.predict(test_dataset)

125/125 [==============================] - 44s 352ms/step


In [50]:
save_preds(pred, "28_model.csv")

28_model.csv Saved!


# model(30)- model(28)에 maxpool 지우기

## Score: 0.25009

In [51]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))


model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_36 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 conv2d_37 (Conv2D)          (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                 

In [52]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-30model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [53]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [54]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("30model.h5")

Epoch 1/100
225/225 [==============================] - 445s 2s/step - loss: 0.6879 - accuracy: 0.5426 - val_loss: 0.6801 - val_accuracy: 0.5732 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 447s 2s/step - loss: 0.6556 - accuracy: 0.6134 - val_loss: 0.6167 - val_accuracy: 0.6796 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 449s 2s/step - loss: 0.5884 - accuracy: 0.6925 - val_loss: 0.5343 - val_accuracy: 0.7368 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 465s 2s/step - loss: 0.5071 - accuracy: 0.7589 - val_loss: 0.4835 - val_accuracy: 0.7568 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 488s 2s/step - loss: 0.4549 - accuracy: 0.7880 - val_loss: 0.4419 - val_accuracy: 0.7904 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 490s 2s/step - loss: 0.4151 - accuracy: 0.8112 - val_loss: 0.3973 - val_accuracy: 0.8284 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 471

In [55]:
pred = model.predict(test_dataset)

125/125 [==============================] - 67s 537ms/step


In [56]:
save_preds(pred, "29_model.csv")

29_model.csv Saved!


# model(31) - model(28)에 maxpool 지우기(2)

## Score: 0.23050

In [61]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))


model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_45 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_46 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_47 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 conv2d_48 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                 

In [62]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-31model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [63]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [64]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("31model.h5")

Epoch 1/100
225/225 [==============================] - 324s 1s/step - loss: 0.6903 - accuracy: 0.5152 - val_loss: 0.6688 - val_accuracy: 0.6140 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 318s 1s/step - loss: 0.6134 - accuracy: 0.6567 - val_loss: 0.5503 - val_accuracy: 0.7124 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 322s 1s/step - loss: 0.5338 - accuracy: 0.7324 - val_loss: 0.4927 - val_accuracy: 0.7612 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 325s 1s/step - loss: 0.4891 - accuracy: 0.7631 - val_loss: 0.4637 - val_accuracy: 0.7824 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 326s 1s/step - loss: 0.4461 - accuracy: 0.7944 - val_loss: 0.4140 - val_accuracy: 0.8028 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 326s 1s/step - loss: 0.4224 - accuracy: 0.8076 - val_loss: 0.3957 - val_accuracy: 0.8224 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 327

In [65]:
pred = model.predict(test_dataset)

125/125 [==============================] - 60s 476ms/step


In [66]:
save_preds(pred, "30_model.csv")

30_model.csv Saved!


# model(32) - model(31)에 early_stopping = 10으로

## Score: 0.24235

In [67]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))


model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_50 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_51 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_52 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 conv2d_53 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                

In [68]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-32model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [69]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [70]:
history = model.fit(train_dataset, epochs = 200, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("32model.h5")

Epoch 1/200
225/225 [==============================] - 403s 2s/step - loss: 0.6676 - accuracy: 0.5869 - val_loss: 0.6376 - val_accuracy: 0.6436 - lr: 0.0010
Epoch 2/200
225/225 [==============================] - 399s 2s/step - loss: 0.5710 - accuracy: 0.7104 - val_loss: 0.4892 - val_accuracy: 0.7636 - lr: 0.0010
Epoch 3/200
225/225 [==============================] - 398s 2s/step - loss: 0.4829 - accuracy: 0.7709 - val_loss: 0.4456 - val_accuracy: 0.7940 - lr: 0.0010
Epoch 4/200
225/225 [==============================] - 399s 2s/step - loss: 0.4402 - accuracy: 0.7972 - val_loss: 0.3976 - val_accuracy: 0.8196 - lr: 0.0010
Epoch 5/200
225/225 [==============================] - 398s 2s/step - loss: 0.4081 - accuracy: 0.8140 - val_loss: 0.3997 - val_accuracy: 0.8092 - lr: 0.0010
Epoch 6/200
225/225 [==============================] - 400s 2s/step - loss: 0.3747 - accuracy: 0.8345 - val_loss: 0.3534 - val_accuracy: 0.8384 - lr: 0.0010
Epoch 7/200
225/225 [==============================] - 401

Epoch 52/200
225/225 [==============================] - 328s 1s/step - loss: 0.1342 - accuracy: 0.9463 - val_loss: 0.2070 - val_accuracy: 0.9160 - lr: 3.9063e-06
Epoch 53/200
225/225 [==============================] - 342s 2s/step - loss: 0.1322 - accuracy: 0.9471 - val_loss: 0.2071 - val_accuracy: 0.9188 - lr: 1.9531e-06
Epoch 54/200
225/225 [==============================] - 333s 1s/step - loss: 0.1280 - accuracy: 0.9482 - val_loss: 0.2057 - val_accuracy: 0.9200 - lr: 1.9531e-06
Epoch 55/200
225/225 [==============================] - 331s 1s/step - loss: 0.1299 - accuracy: 0.9489 - val_loss: 0.2031 - val_accuracy: 0.9216 - lr: 1.9531e-06
Epoch 56/200
225/225 [==============================] - 332s 1s/step - loss: 0.1293 - accuracy: 0.9492 - val_loss: 0.2056 - val_accuracy: 0.9228 - lr: 9.7656e-07


In [71]:
pred = model.predict(test_dataset)

125/125 [==============================] - 52s 415ms/step


In [72]:
save_preds(pred, "31_model.csv")

31_model.csv Saved!


# model(33) - model(28)에 early_stopping = 10으로

## Score: 0.22116

In [77]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_61 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_62 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_63 (Conv2D)          (None, 32, 32, 64)      

In [78]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-32model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [79]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [80]:
history = model.fit(train_dataset, epochs = 200, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("32model.h5")

Epoch 1/200
225/225 [==============================] - 247s 1s/step - loss: 0.6701 - accuracy: 0.5752 - val_loss: 0.6089 - val_accuracy: 0.6608 - lr: 0.0010
Epoch 2/200
225/225 [==============================] - 230s 1s/step - loss: 0.5709 - accuracy: 0.7022 - val_loss: 0.5145 - val_accuracy: 0.7392 - lr: 0.0010
Epoch 3/200
225/225 [==============================] - 230s 1s/step - loss: 0.4986 - accuracy: 0.7586 - val_loss: 0.4289 - val_accuracy: 0.7940 - lr: 0.0010
Epoch 4/200
225/225 [==============================] - 230s 1s/step - loss: 0.4333 - accuracy: 0.8027 - val_loss: 0.3923 - val_accuracy: 0.8228 - lr: 0.0010
Epoch 5/200
225/225 [==============================] - 230s 1s/step - loss: 0.3822 - accuracy: 0.8303 - val_loss: 0.3540 - val_accuracy: 0.8356 - lr: 0.0010
Epoch 6/200
225/225 [==============================] - 231s 1s/step - loss: 0.3573 - accuracy: 0.8448 - val_loss: 0.3385 - val_accuracy: 0.8460 - lr: 0.0010
Epoch 7/200
225/225 [==============================] - 230

In [81]:
pred = model.predict(test_dataset)

125/125 [==============================] - 47s 372ms/step


In [82]:
save_preds(pred, "31_model.csv")

31_model.csv Saved!


# model(34) - model(28)에 conv2d 추가

## Score: 0.20791

In [83]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))

model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same"))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_65 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_66 (Conv2D)          (None, 128, 128, 8)       584       
                                                                 
 conv2d_67 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_48 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_68 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                

In [84]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-34model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [85]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [86]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("34model.h5")

Epoch 1/100
225/225 [==============================] - 275s 1s/step - loss: 0.6595 - accuracy: 0.5925 - val_loss: 0.6057 - val_accuracy: 0.6760 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 263s 1s/step - loss: 0.5635 - accuracy: 0.7072 - val_loss: 0.4909 - val_accuracy: 0.7592 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 264s 1s/step - loss: 0.4959 - accuracy: 0.7607 - val_loss: 0.4450 - val_accuracy: 0.7824 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 266s 1s/step - loss: 0.4423 - accuracy: 0.7948 - val_loss: 0.4030 - val_accuracy: 0.8156 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 266s 1s/step - loss: 0.4082 - accuracy: 0.8156 - val_loss: 0.3850 - val_accuracy: 0.8284 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 266s 1s/step - loss: 0.3792 - accuracy: 0.8328 - val_loss: 0.3584 - val_accuracy: 0.8460 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 267

In [87]:
pred = model.predict(test_dataset)

125/125 [==============================] - 55s 435ms/step


In [88]:
save_preds(pred, "34_model.csv")

34_model.csv Saved!


# model(35) - model(28)에 conv2d 추가, maxpool추가

## Score: 0.23956

In [89]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))

model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_71 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_72 (Conv2D)          (None, 128, 128, 8)       584       
                                                                 
 max_pooling2d_52 (MaxPoolin  (None, 64, 64, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_73 (Conv2D)          (None, 64, 64, 16)        1168      
                                                                 
 max_pooling2d_53 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_74 (Conv2D)          (None, 32, 32, 32)      

In [90]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-35model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [91]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [92]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("35model.h5")

Epoch 1/100
225/225 [==============================] - 234s 1s/step - loss: 0.6565 - accuracy: 0.5987 - val_loss: 0.5748 - val_accuracy: 0.7016 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 230s 1s/step - loss: 0.5461 - accuracy: 0.7236 - val_loss: 0.4895 - val_accuracy: 0.7744 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 231s 1s/step - loss: 0.4726 - accuracy: 0.7767 - val_loss: 0.4465 - val_accuracy: 0.7912 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 231s 1s/step - loss: 0.4300 - accuracy: 0.8030 - val_loss: 0.4471 - val_accuracy: 0.7960 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 231s 1s/step - loss: 0.4069 - accuracy: 0.8172 - val_loss: 0.3767 - val_accuracy: 0.8260 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 231s 1s/step - loss: 0.3633 - accuracy: 0.8389 - val_loss: 0.3638 - val_accuracy: 0.8268 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 232

In [93]:
pred = model.predict(test_dataset)

125/125 [==============================] - 40s 321ms/step


In [94]:
save_preds(pred, "35_model.csv")

35_model.csv Saved!


# model(36) - model(28)에 conv2d 추가

## Score: 0.23347

In [95]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_77 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_78 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_79 (Conv2D)          (None, 64, 64, 16)        2320      
                                                                 
 conv2d_80 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                

In [96]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-36model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [97]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [98]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("36model.h5")

Epoch 1/100
225/225 [==============================] - 309s 1s/step - loss: 0.6675 - accuracy: 0.5794 - val_loss: 0.6239 - val_accuracy: 0.6364 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 308s 1s/step - loss: 0.5814 - accuracy: 0.6894 - val_loss: 0.5333 - val_accuracy: 0.7312 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 308s 1s/step - loss: 0.5036 - accuracy: 0.7580 - val_loss: 0.4874 - val_accuracy: 0.7668 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 309s 1s/step - loss: 0.4596 - accuracy: 0.7857 - val_loss: 0.4369 - val_accuracy: 0.7948 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 308s 1s/step - loss: 0.4315 - accuracy: 0.8042 - val_loss: 0.4197 - val_accuracy: 0.8156 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 309s 1s/step - loss: 0.4000 - accuracy: 0.8228 - val_loss: 0.4094 - val_accuracy: 0.8152 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 308

In [99]:
pred = model.predict(test_dataset)

125/125 [==============================] - 50s 396ms/step


In [100]:
save_preds(pred, "36_model.csv")

36_model.csv Saved!


# model(37) - model(28)에 conv2d 추가, maxpool추가

## Score: 0.21295

In [101]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_83 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_84 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_61 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_85 (Conv2D)          (None, 64, 64, 16)        2320      
                                                                 
 max_pooling2d_62 (MaxPoolin  (None, 32, 32, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_86 (Conv2D)          (None, 32, 32, 32)      

In [102]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-37model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [103]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [104]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("37model.h5")

Epoch 1/100
225/225 [==============================] - 263s 1s/step - loss: 0.6545 - accuracy: 0.5924 - val_loss: 0.5751 - val_accuracy: 0.6824 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 260s 1s/step - loss: 0.5652 - accuracy: 0.7055 - val_loss: 0.5154 - val_accuracy: 0.7508 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 259s 1s/step - loss: 0.4869 - accuracy: 0.7661 - val_loss: 0.4322 - val_accuracy: 0.7936 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 260s 1s/step - loss: 0.4203 - accuracy: 0.8079 - val_loss: 0.4132 - val_accuracy: 0.8052 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 259s 1s/step - loss: 0.3734 - accuracy: 0.8328 - val_loss: 0.3626 - val_accuracy: 0.8320 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 260s 1s/step - loss: 0.3312 - accuracy: 0.8564 - val_loss: 0.3079 - val_accuracy: 0.8640 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 242

In [105]:
pred = model.predict(test_dataset)

125/125 [==============================] - 43s 341ms/step


In [106]:
save_preds(pred, "37_model.csv")

37_model.csv Saved!


# model(38) - model(28)에 leaky_relu 추가

## Score: 0.23046

In [107]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_89 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_90 (Conv2D)          (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_66 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_91 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_67 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_92 (Conv2D)          (None, 32, 32, 64)      

In [108]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-38model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [109]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [110]:
history = model.fit(train_dataset, epochs = 100, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("38model.h5")

Epoch 1/100
225/225 [==============================] - 245s 1s/step - loss: 0.6750 - accuracy: 0.5659 - val_loss: 0.6198 - val_accuracy: 0.6572 - lr: 0.0010
Epoch 2/100
225/225 [==============================] - 232s 1s/step - loss: 0.5634 - accuracy: 0.7143 - val_loss: 0.5345 - val_accuracy: 0.7208 - lr: 0.0010
Epoch 3/100
225/225 [==============================] - 231s 1s/step - loss: 0.4655 - accuracy: 0.7801 - val_loss: 0.4267 - val_accuracy: 0.8080 - lr: 0.0010
Epoch 4/100
225/225 [==============================] - 242s 1s/step - loss: 0.4077 - accuracy: 0.8161 - val_loss: 0.3814 - val_accuracy: 0.8236 - lr: 0.0010
Epoch 5/100
225/225 [==============================] - 255s 1s/step - loss: 0.3599 - accuracy: 0.8448 - val_loss: 0.3325 - val_accuracy: 0.8592 - lr: 0.0010
Epoch 6/100
225/225 [==============================] - 254s 1s/step - loss: 0.3277 - accuracy: 0.8584 - val_loss: 0.3240 - val_accuracy: 0.8556 - lr: 0.0010
Epoch 7/100
225/225 [==============================] - 236

In [111]:
pred = model.predict(test_dataset)

125/125 [==============================] - 50s 397ms/step


In [112]:
save_preds(pred, "38_model.csv")

38_model.csv Saved!


# model(39) - model(28)에 dropout추가 + earlystopping = 10

## Score: 0.20008

In [117]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_99 (Conv2D)          (None, 128, 128, 8)       224       
                                                                 
 conv2d_100 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_74 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_101 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_75 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 dropout_53 (Dropout)        (None, 32, 32, 32)      

In [118]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-39model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [119]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [120]:
history = model.fit(train_dataset, epochs = 200, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("39model.h5")

Epoch 1/200
225/225 [==============================] - 262s 1s/step - loss: 0.6746 - accuracy: 0.5729 - val_loss: 0.6000 - val_accuracy: 0.6836 - lr: 0.0010
Epoch 2/200
225/225 [==============================] - 261s 1s/step - loss: 0.5593 - accuracy: 0.7140 - val_loss: 0.4943 - val_accuracy: 0.7612 - lr: 0.0010
Epoch 3/200
225/225 [==============================] - 245s 1s/step - loss: 0.4979 - accuracy: 0.7599 - val_loss: 0.4476 - val_accuracy: 0.7820 - lr: 0.0010
Epoch 4/200
225/225 [==============================] - 246s 1s/step - loss: 0.4466 - accuracy: 0.7948 - val_loss: 0.4211 - val_accuracy: 0.8080 - lr: 0.0010
Epoch 5/200
225/225 [==============================] - 273s 1s/step - loss: 0.4033 - accuracy: 0.8176 - val_loss: 0.3994 - val_accuracy: 0.8116 - lr: 0.0010
Epoch 6/200
225/225 [==============================] - 282s 1s/step - loss: 0.3796 - accuracy: 0.8308 - val_loss: 0.4039 - val_accuracy: 0.8192 - lr: 0.0010
Epoch 7/200
225/225 [==============================] - 292

In [121]:
pred = model.predict(test_dataset)

125/125 [==============================] - 51s 406ms/step


In [122]:
save_preds(pred, "39_model.csv")

39_model.csv Saved!


# model(40) - model(39)에 activation = leaky_relu로 변경

## Score: 0.20807

In [123]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size = 3, activation = "leaky_relu", padding = "same",
                              input_shape = (128,128, 3)))


model.add(keras.layers.Conv2D(16, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Conv2D(32, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Conv2D(128, kernel_size = 3, activation = "leaky_relu", padding = "same"))
model.add(keras.layers.MaxPool2D(2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(0.25))

model.add(keras.layers.Dense(16, activation = "leaky_relu"))

model.add(keras.layers.Dense(32, activation = "leaky_relu"))

model.add(keras.layers.Dropout(0.4))

model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_104 (Conv2D)         (None, 128, 128, 8)       224       
                                                                 
 conv2d_105 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 max_pooling2d_78 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_106 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_79 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 dropout_57 (Dropout)        (None, 32, 32, 32)      

In [124]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("./model/best-cnn-40model.h5", save_best_only = True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = .5, patience = 3)

In [125]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "accuracy")

In [126]:
history = model.fit(train_dataset, epochs = 200, validation_data = val_dataset,
                    callbacks = [checkpoint_cb, early_stopping_cb, reducelr_cb])
model.save_weights("40model.h5")

Epoch 1/200
225/225 [==============================] - 269s 1s/step - loss: 0.6624 - accuracy: 0.5893 - val_loss: 0.6317 - val_accuracy: 0.6420 - lr: 0.0010
Epoch 2/200
225/225 [==============================] - 270s 1s/step - loss: 0.5732 - accuracy: 0.6991 - val_loss: 0.5187 - val_accuracy: 0.7380 - lr: 0.0010
Epoch 3/200
225/225 [==============================] - 272s 1s/step - loss: 0.5057 - accuracy: 0.7552 - val_loss: 0.4743 - val_accuracy: 0.7824 - lr: 0.0010
Epoch 4/200
225/225 [==============================] - 272s 1s/step - loss: 0.4583 - accuracy: 0.7858 - val_loss: 0.4260 - val_accuracy: 0.8088 - lr: 0.0010
Epoch 5/200
225/225 [==============================] - 272s 1s/step - loss: 0.4222 - accuracy: 0.8068 - val_loss: 0.3930 - val_accuracy: 0.8200 - lr: 0.0010
Epoch 6/200
225/225 [==============================] - 272s 1s/step - loss: 0.3918 - accuracy: 0.8261 - val_loss: 0.3906 - val_accuracy: 0.8240 - lr: 0.0010
Epoch 7/200
225/225 [==============================] - 274

In [127]:
pred = model.predict(test_dataset)

125/125 [==============================] - 47s 377ms/step


In [128]:
save_preds(pred, "40_model.csv")

40_model.csv Saved!
